In [6]:
from flask import Flask, render_template, request
import pickle
import cv2
from skimage.transform import resize
from skimage.feature import hog
import numpy as np
import os

app = Flask(__name__)

# Load the saved SVM model
model_filename = 'svm_model.pkl'
with open(model_filename, 'rb') as model_file:
    loaded_model = pickle.load(model_file)

# Function to extract HOG features
def extract_hog_features(images, target_size=(128, 128)):
    hog_features = []
    for image in images:
        resized_image = resize(image, target_size, anti_aliasing=True)
        features = hog(resized_image, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(1, 1), block_norm='L2-Hys', visualize=False, feature_vector=True)
        hog_features.append(features)
    return np.vstack(hog_features)

# Function to preprocess the uploaded image
def preprocess_image(image):
    if image is not None:
        grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        enhanced_image = cv2.equalizeHist(grayscale_image)
        return [enhanced_image]
    else:
        return None

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        # Check if the post request has the file part
        if 'file' not in request.files:
            return render_template('index.html', error='No file part')

        file = request.files['file']

        # If the user does not select a file, browser submits an empty file without a filename
        if file.filename == '':
            return render_template('index.html', error='No selected file')

        # If the file exists and is allowed, process the image
        if file and allowed_file(file.filename):
            image = cv2.imdecode(np.fromstring(file.read(), np.uint8), cv2.IMREAD_UNCHANGED)
            processed_image = preprocess_image(image)

            if processed_image is not None:
                # Extract HOG features for the uploaded image
                X_uploaded_hog = extract_hog_features(processed_image)

                # Prediction on the uploaded image
                predicted_label = loaded_model.predict(X_uploaded_hog)[0]

                return render_template('index.html', prediction=predicted_label)
            else:
                return render_template('index.html', error='Failed to process the image')

    return render_template('index.html', error=None)

# Function to check if the file extension is allowed
def allowed_file(filename):
    ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Dec/2023 21:11:09] "GET / HTTP/1.1" 200 -
C:\Users\FahadKamran\AppData\Local\Temp\ipykernel_3644\4014821166.py:49: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  image = cv2.imdecode(np.fromstring(file.read(), np.uint8), cv2.IMREAD_UNCHANGED)
127.0.0.1 - - [03/Dec/2023 21:11:36] "POST / HTTP/1.1" 200 -
C:\Users\FahadKamran\AppData\Local\Temp\ipykernel_3644\4014821166.py:49: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  image = cv2.imdecode(np.fromstring(file.read(), np.uint8), cv2.IMREAD_UNCHANGED)
127.0.0.1 - - [03/Dec/2023 21:11:53] "POST / HTTP/1.1" 200 -
